In [21]:
import os 
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import numpy as np
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt


In [23]:

# Load the dataset
os.chdir('/Users/mary/Downloads/data cleannnn.csv') # this will be the path to your local directory where the repository is saved
file_path = 'data/df_arabica_clean.csv'  
data = pd.read_csv(file_path)
data.head()


FileNotFoundError: [Errno 2] No such file or directory: 'data cleannnn.csv'

In [25]:
# Select numerical columns for clustering
features = ['Aroma', 'Flavor', 'Aftertaste', 'Acidity', 'Body','Balance', 'Overall']

# Extract relevant data
X = data[features].dropna()
X.head()
# Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


NameError: name 'data' is not defined

In [ ]:
# using Elbow method adn silhouette score to find the optimal number of clusters
inertia = []
silhouette_scores = []
K = range(2, 11)  # Silhouette score requires at least 2 clusters

for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_scaled)
    inertia.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X_scaled, kmeans.labels_))

# Plot Elbow Curve and Silhouette Scores
fig, ax1 = plt.subplots(figsize=(8, 5))
ax2 = ax1.twinx()

ax1.plot(K, inertia, 'bo-', label='Inertia')
ax2.plot(K, silhouette_scores, 'r^-', label='Silhouette Score')

ax1.set_xlabel('Number of Clusters (k)')
ax1.set_ylabel('Inertia', color='b')
ax2.set_ylabel('Silhouette Score', color='r')
plt.title('Elbow Method and Silhouette Analysis for Optimal k')
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')
plt.show()



In [ ]:
# Based on the elbow plot, let's choose k = 3 as the optimal number of clusters trying 4 also
# Set the number of clusters for k=3 and k=4
kmeans_3 = KMeans(n_clusters=3, random_state=42)
kmeans_4 = KMeans(n_clusters=4, random_state=42)

# Fit the models to the scaled data
kmeans_3.fit(X_scaled)
kmeans_4.fit(X_scaled)

# Get the cluster labels for each k
data['Cluster_k3'] = kmeans_3.labels_
data['Cluster_k4'] = kmeans_4.labels_

# Check the first few rows of the dataframe with both cluster labels
data.head()



In [ ]:
from sklearn.decomposition import PCA

# Function to visualize clusters for a given k
def plot_clusters(k, X_scaled):
    # Fit KMeans with specified number of clusters
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_scaled)
    labels = kmeans.labels_
    
    # Reduce to 2D for visualization
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X_scaled)
    
    # Plot clusters
    plt.figure(figsize=(8, 6))
    plt.scatter(X_pca[:, 0], X_pca[:, 1], c=labels, cmap='viridis', marker='o', edgecolor='k', s=50)
    plt.title(f'Visualization of Clusters for k = {k}')
    plt.xlabel('PCA Component 1')
    plt.ylabel('PCA Component 2')
    plt.colorbar(label='Cluster Label')
    plt.show()

# Visualize clusters for k = 3
plot_clusters(3, X_scaled)

# Visualize clusters for k = 4
plot_clusters(4, X_scaled)



In [ ]:
# Create a DataFrame from the scaled data and add the cluster labels
X_df = pd.DataFrame(X, columns=features)  # Assuming 'features' is your list of selected feature names
X_df['Cluster_k3'] = kmeans_3.labels_
X_df['Cluster_k4'] = kmeans_4.labels_


# Display cluster means for k = 3
cluster_means_k3 = X_df.groupby('Cluster_k3').mean()
print("Cluster Means for k = 3:")
print(cluster_means_k3)

# Display cluster means for k = 4
cluster_means_k4 = X_df.groupby('Cluster_k4').mean()
print("\nCluster Means for k = 4:")
print(cluster_means_k4)




In [ ]:

# Reduce the scaled data to 2D for plotting
pca = PCA(n_components=2)
data_pca = pca.fit_transform(X_scaled)

def plot_decision_boundary(k, data_pca):
    # Generate a mesh grid for the decision boundary
    x_min, x_max = data_pca[:, 0].min() - 1, data_pca[:, 0].max() + 1
    y_min, y_max = data_pca[:, 1].min() - 1, data_pca[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02),
                         np.arange(y_min, y_max, 0.02))
    
    # Fit KMeans on the PCA-reduced data
    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(data_pca)
    Z = kmeans.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
    cluster_labels = kmeans.labels_
    
    # Plot decision boundaries and data points
    plt.figure(figsize=(10, 8))
    plt.contourf(xx, yy, Z, cmap='Pastel2', alpha=0.3)  # Decision boundaries
    plt.scatter(data_pca[:, 0], data_pca[:, 1], c=cluster_labels, cmap='viridis', edgecolor='k', s=50)  # Data points
    
    # Plot centroids
    centroids = kmeans.cluster_centers_
    plt.scatter(centroids[:, 0], centroids[:, 1], s=300, c='red', marker='X', label='Centroids')
    
    # Labels and title
    plt.xlabel('PCA Component 1')
    plt.ylabel('PCA Component 2')
    plt.title(f'Decision Boundaries Between Clusters in PCA-Reduced 2D Space (k = {k})')
    plt.legend()
    plt.show()

# Plot for k = 3
plot_decision_boundary(3, data_pca)

# Plot for k = 4
plot_decision_boundary(4, data_pca)


In [ ]:


# Function to fit GMM and plot decision boundaries
def plot_gmm_decision_boundary(k, data_pca):
    # Fit the GMM model
    gmm = GaussianMixture(n_components=k, random_state=0)
    gmm.fit(data_pca)
    labels = gmm.predict(data_pca)
    
    # Generate a mesh grid for decision boundaries
    x_min, x_max = data_pca[:, 0].min() - 1, data_pca[:, 0].max() + 1
    y_min, y_max = data_pca[:, 1].min() - 1, data_pca[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02),
                         np.arange(y_min, y_max, 0.02))
    
    # Predict on the mesh grid for decision boundary visualization
    Z = gmm.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

    # Plot decision boundaries and data points
    plt.figure(figsize=(10, 8))
    plt.contourf(xx, yy, Z, cmap='Pastel2', alpha=0.3)  # Decision boundary regions
    plt.scatter(data_pca[:, 0], data_pca[:, 1], c=labels, cmap='viridis', edgecolor='k', s=50)  # Data points
    
    # Plot GMM component means (centroids)
    centroids = gmm.means_
    plt.scatter(centroids[:, 0], centroids[:, 1], s=300, c='red', marker='X', label='GMM Centroids')
    
    # Labels and title
    plt.xlabel('PCA Component 1')
    plt.ylabel('PCA Component 2')
    plt.title(f'GMM Decision Boundaries in PCA-Reduced 2D Space (k = {k})')
    plt.legend()
    plt.show()

# Plot for GMM with k = 3
plot_gmm_decision_boundary(3, data_pca)

# Plot for GMM with k = 4
plot_gmm_decision_boundary(4, data_pca)


In [ ]:
def plot_tsne_clusters(data, k):
    """
    Plots t-SNE clusters with KMeans centroids and convex hulls.
    
    Parameters:
    - data: numpy array or pandas DataFrame, preprocessed data to be reduced with t-SNE.
    - k: int, number of clusters for KMeans.
    """
    # Reduce data to 2D using t-SNE
    tsne = TSNE(n_components=2, random_state=0)
    data_tsne = tsne.fit_transform(data)

    # Fit KMeans on the t-SNE-transformed data with specified k
    kmeans = KMeans(n_clusters=k, random_state=0)
    cluster_labels = kmeans.fit_predict(data_tsne)
    centroids = kmeans.cluster_centers_

    # Plot t-SNE clusters with centroids
    plt.figure(figsize=(10, 10))
    plt.scatter(data_tsne[:, 0], data_tsne[:, 1], c=cluster_labels, cmap='viridis', edgecolor='k', s=50)
    plt.scatter(centroids[:, 0], centroids[:, 1], s=300, c='red', marker='X', label='Centroids')

    # Draw convex hulls around each cluster for boundary-like effect
    for i in range(k):
        points = data_tsne[cluster_labels == i]
        hull = ConvexHull(points)
        plt.plot(points[hull.vertices, 0], points[hull.vertices, 1], 'k-', lw=2)

    # Labels and title
    plt.xlabel('t-SNE Component 1')
    plt.ylabel('t-SNE Component 2')
    plt.title(f't-SNE Cluster Visualization for k={k} with Convex Hulls')
    plt.legend()
    plt.show()

In [ ]:
plot_tsne_clusters(data_scaled, k=3)  # For k=3
plot_tsne_clusters(data_scaled, k=4)  # For k=4

In [ ]:
def plot_umap_decision_boundaries(data, k):
    """
    Reduces data to 2D using UMAP, applies KMeans clustering, and plots decision boundaries.
    
    Parameters:
    - data: numpy array or pandas DataFrame, preprocessed data to be reduced with UMAP.
    - k: int, number of clusters for KMeans.
    """
    # Step 1: Apply UMAP for 2D dimensionality reduction
    umap_reducer = umap.UMAP(n_components=2, random_state=0)
    data_umap = umap_reducer.fit_transform(data).astype(np.float64)  # Convert to float64

    # Step 2: Fit KMeans on the UMAP-reduced data with specified k
    kmeans = KMeans(n_clusters=k, random_state=0)
    cluster_labels = kmeans.fit_predict(data_umap)
    centroids = kmeans.cluster_centers_

    # Step 3: Create a mesh grid for plotting decision boundaries based on UMAP data range
    x_min, x_max = data_umap[:, 0].min() - 1, data_umap[:, 0].max() + 1
    y_min, y_max = data_umap[:, 1].min() - 1, data_umap[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02), np.arange(y_min, y_max, 0.02))

    # Ensure mesh grid is in float64 format
    mesh_points = np.c_[xx.ravel(), yy.ravel()].astype(np.float64)

    # Predict the cluster for each point on the grid
    Z = kmeans.predict(mesh_points).reshape(xx.shape)

    # Step 4: Plot decision boundaries, clusters, and centroids
    plt.figure(figsize=(10, 8))
    plt.contourf(xx, yy, Z, cmap='Pastel2', alpha=0.3)
    plt.scatter(data_umap[:, 0], data_umap[:, 1], c=cluster_labels, cmap='viridis', edgecolor='k', s=50)
    plt.scatter(centroids[:, 0], centroids[:, 1], s=300, c='red', marker='X', label='Centroids')

    # Label and title
    plt.xlabel('UMAP Component 1')
    plt.ylabel('UMAP Component 2')
    plt.title(f'UMAP-Based Decision Boundaries for k={k}')
    plt.legend()
    plt.show()



In [ ]:
plot_umap_decision_boundaries(data_scaled, k=3)  # For k=3
plot_umap_decision_boundaries(data_scaled, k=4)  # For k=4